In [11]:
from function import *

In [12]:
print(os.getcwd())
os.chdir("/home/wmbio/WORK/gitworking/Dependency_prediction/")
print(os.getcwd())

/home/wmbio/WORK/gitworking/Dependency_prediction
/home/wmbio/WORK/gitworking/Dependency_prediction


In [13]:
with tf.device('/cpu:0'):
    model_name = "model_full_0801"
    # model_saved = tf.keras.models.load_model("prediction/custom_model/%s.h5" % model_name)
    model_saved = models.load_model("prediction/custom_model/%s.h5" % model_name)
    
    # model_name = "model_paper"
    # model_saved = models.load_model("prediction/model/%s.h5" % model_name)
# model_paper is the full 4-omics DeepDEP model used in the paper
# user can choose from single-omics, 2-omics, or full DeepDEP models from the
# /data/full_results_models_paper/models/ directory

* **Predict Set load**

In [23]:
"""
exp..mut..cna..meth
exp..mut..cna
exp..mut

mut..cna..meth
"""
PREDICT_PATH = "preprocessing/PREDICTION/2022-08-01/"

In [24]:
barcode = "exp..mut..cna..meth"

In [15]:
# load TCGA genomics data and gene fingerprints
data_mut, data_labels_mut, sample_names_mut, gene_names_mut = load_data_prediction(PREDICT_PATH + "predict_mut_" + barcode + ".txt")
data_exp, data_labels_exp, sample_names_exp, gene_names_exp = load_data_prediction(PREDICT_PATH + "predict_exp_" + barcode + ".txt")
data_cna, data_labels_cna, sample_names_cna, gene_names_cna = load_data_prediction(PREDICT_PATH + "predict_cna_" + barcode + ".txt")
data_meth, data_labels_meth, sample_names_meth, gene_names_meth = load_data_prediction(PREDICT_PATH + "predict_meth_prediction.txt")
data_fprint_DepOIs, data_labels_fprint, gene_names_fprint, function_names_fprint = load_data_prediction(PREDICT_PATH + "predict_fingerprint_prediction.txt")
print("\n\nDatasets successfully loaded.\n\n")



Datasets successfully loaded.




In [16]:
pd.DataFrame(data_exp)

,0,1,2,3,4,5,6,7,8,9,...,6006,6007,6008,6009,6010,6011,6012,6013,6014,6015
0,5.5948,1.4542,0.0144,0.2750,3.5435,3.3420,0.7137,0.2388,0.6508,3.3674,...,0.0141,3.7371,0.1948,2.6169,0.5883,3.0723,1.4847,4.65,0.4107,0.4151
1,5.1914,0.1890,0.0000,3.3505,2.7485,3.2373,1.1309,3.6904,3.4944,5.4912,...,0.0141,3.7371,0.1948,2.6169,0.5883,3.0723,1.4847,4.65,0.4107,0.4151
2,5.3618,0.2265,0.0144,0.0286,0.0144,1.2869,0.0286,1.8520,1.2869,4.7522,...,0.0141,3.7371,0.1948,2.6169,0.5883,3.0723,1.4847,4.65,0.4107,0.4151


In [17]:
batch_size = 300
first_to_predict = 3
# predict the first 10 samples for DEMO ONLY, for all samples please substitute 10 by data_mut_tcga.shape[0]
# prediction results of all 8238 TCGA samples can be found in /data/full_results_models_paper/predictions/

In [19]:
with tf.device('/cpu:0'):
    t = time.time()
    data_pred = np.zeros((first_to_predict, data_fprint_DepOIs.shape[0]))
    for z in np.arange(0, first_to_predict):
        data_pred_tmp = model_saved.predict([
            data_mut[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_exp[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_cna[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_meth[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_fprint_DepOIs
        ], 
            batch_size=batch_size, verbose=0)
        data_pred[z] = np.transpose(data_pred_tmp)
        print("Unscreended sample %d predicted..." % z)

Unscreended sample 0 predicted...
Unscreended sample 1 predicted...
Unscreended sample 2 predicted...


In [20]:
# write prediction results to txt
data_pred_df = pd.DataFrame(data=np.transpose(data_pred), index=gene_names_fprint, columns=sample_names_mut[0:first_to_predict])

In [21]:
data_pred_df

,IPC298,LOUNH91,697
AARS2,-0.628335,-0.323625,-0.627580
AASDHPPT,-0.598678,-0.237529,-0.553973
ABCE1,-1.851579,-1.848119,-1.800221
ABCG1,-0.104803,-0.181353,-0.154147
ABHD11,-0.519524,-0.312635,-0.347383
...,...,...,...
ZNF384,-0.027886,-0.119026,-0.083906
ZNF407,-0.543440,-0.716532,-0.587453
ZNF521,-0.020093,-0.038354,-0.012089
ZNHIT2,-1.710741,-1.739170,-1.765369


In [ ]:
model_saved.predict([
            # data_mut[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_exp[np.repeat(z, data_fprint_DepOIs.shape[0])],
            # data_cna[np.repeat(z, data_fprint_DepOIs.shape[0])],
            # data_meth[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_fprint_DepOIs
        ], 
            batch_size=batch_size, verbose=0)

In [ ]:
model_saved.predict([
            # data_mut[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_exp[np.repeat(z, data_fprint_DepOIs.shape[0])],
            # data_cna[np.repeat(z, data_fprint_DepOIs.shape[0])],
            # data_meth[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_fprint_DepOIs
        ], 
            batch_size=batch_size, verbose=0)

In [ ]:
data_pred_df.to_csv('custom_mut_exp.csv')

In [ ]:
# pd.DataFrame.to_csv(data_pred_df, path_or_buf="/results/predictions/tcga_predicted_data_%s_demo.txt" % model_name, sep='\t', index_label='CRISPR_GENE', float_format='%.4f')
# print("\n\nPrediction completed in %.1f mins.\nResults saved in /results/predictions/tcga_predicted_data_%s_demo.txt\n\n" % (
#     (time.time()-t)/60, model_name))